In [6]:
from bs4 import BeautifulSoup 
import urllib.request
import ssl
from dotenv import load_dotenv
import openai
import os
import requests
import warnings
warnings.filterwarnings("ignore")
from tqdm import tqdm
import pandas as pd
import json
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re
from sqlalchemy import inspect
import sqlalchemy
from sqlalchemy import text 
import dspy
from termcolor import colored

In [19]:
import os

TOKEN="TOKEN"
ENDPOINT = 'https://models.inference.ai.azure.com'
HF_TOKEN = 'TOKEN'

os.environ['HUGGINGFACE_TOKEN']= HF_TOKEN
os.environ['GITHUB_TOKEN']= TOKEN
os.environ["OPENAI_API_KEY"] = TOKEN
os.environ['OPENAI_API_BASE'] = ENDPOINT

In [20]:
# Share generation args between models
generation_args = {
    "temperature":0,
    "max_tokens":500,
    "stop":"\n\n",
    "model_type":"chat",
    "n": 1
}
# Model specific args
model_info = {
    "gpt-4o": {"model": "openai/gpt-4o", "api_base": ENDPOINT, "api_key":TOKEN},
    "gpt-4": {"model": "openai/gpt-3.5-turbo", "api_base": ENDPOINT, "api_key":TOKEN}
}

In [21]:
# Set up the models
# lm = StarlingLM(**model_info["starling"], **generation_args)
lm = dspy.LM(**model_info["gpt-4o"], **generation_args)
evaluator_lm = dspy.LM(**model_info["gpt-4o"], **generation_args)

dspy.configure(lm=lm)

In [28]:
dspy.LM?

Init signature:
dspy.LM(
    model: str,
    model_type: Literal['chat', 'text'] = 'chat',
    temperature: float = 0.0,
    max_tokens: int = 1000,
    cache: bool = True,
    callbacks: Optional[List[dspy.utils.callback.BaseCallback]] = None,
    num_retries: int = 3,
    provider=None,
    finetuning_model: Optional[str] = None,
    launch_kwargs: Optional[dict[str, Any]] = None,
    **kwargs,
)
Docstring:      A language model supporting chat or text completion requests for use with DSPy modules.
Init docstring:
Create a new language model instance for use with DSPy modules and programs.

Args:
    model: The model to use. This should be a string of the form ``"llm_provider/llm_name"``
           supported by LiteLLM. For example, ``"openai/gpt-4o"``.
    model_type: The type of the model, either ``"chat"`` or ``"text"``.
    temperature: The sampling temperature to use when generating responses.
    max_tokens: The maximum number of tokens to generate per response.
    cache: Whet

In [1]:
# prompt: import a sqlite model from local env

import sqlite3

conn = sqlite3.connect('employees.db')  # Replace 'your_database.db' with the actual file name
cursor = conn.cursor()

# Example query (replace with your actual queries)
cursor.execute("SELECT * FROM employee_data WHERE StartDate > '2020-01-01' AND PerformanceScore = 'Fully Meets';")
tables = cursor.fetchall()
print(tables)

# ... (rest of your code to interact with the database)

conn.close()

[(1499, 'Abigayle', 'Mccann', '26-Feb-22', '16-Oct-22', 'Production Technician I', 'Lori Lopez DVM', 'abigayle.mccann@bilearner.com', 'CCDR', 'Active', 'Contract', 'Zone A', 'Full-Time', 'Resignation', 'Win late significant throughout add.', 'Production', 'General - Con', '27/04/1975', 'MA', 'Operator', 'Female', 94816, 'Other', 'Married', 'Fully Meets', 3), (1704, 'Mckenzie', 'Kim', '26-Nov-20', None, 'CIO', 'Rachel Thomas', 'mckenzie.kim@bilearner.com', 'CCDR', 'Active', 'Part-Time', 'Zone C', 'Part-Time', 'Unk', None, 'IT/IS', 'Project Management - Con', '21/06/1994', 'MA', 'Project Manager', 'Female', 59091, 'Hispanic', 'Widowed', 'Fully Meets', 3), (1738, 'Phil', 'Close', '30-May-23', '05-Jul-23', 'Production Technician II', 'Tami Fuentes', 'phil.close@bilearner.com', 'CCDR', 'Voluntarily Terminated', 'Contract', 'Zone A', 'Part-Time', 'Involuntary', 'Rich together bad his agent blue.', 'Production', 'General - Eng', '17/11/1983', 'MA', 'Coordinator', 'Male', 2169, 'White', 'Singl

In [30]:
evaluation_set = [
    {
        "question": "What are the names of employees working in sales?",
        "gold_query": "SELECT FirstName, LastName FROM employee_data WHERE BusinessUnit='Sales';"
    },
    {
        "question": "What is the email address of the employee with EmpID 200?",
        "gold_query": "SELECT ADEmail FROM employee_data WHERE EmpID=200;"
    },
    {
        "question": "Retrieve the job title of employees under supervisor 'Jane Doe'.",
        "gold_query": "SELECT Title FROM employee_data WHERE Supervisor='Jane Doe';"
    },
    {
        "question": "How many employees are currently active?",
        "gold_query": "SELECT COUNT(*) FROM employee_data WHERE EmployeeStatus='Active';"
    },
    {
        "question": "List all terminated employees along with their termination reasons.",
        "gold_query": "SELECT FirstName, LastName, TerminationDescription FROM employee_data WHERE TerminationType IS NOT NULL;"
    },
    {
        "question": "What is the desired salary of applicants for the Software Engineer role?",
        "gold_query": "SELECT DesiredSalary FROM recruitment_data WHERE Title='Software Engineer';"
    },
    {
        "question": "How many applicants have more than 5 years of experience?",
        "gold_query": "SELECT COUNT(*) FROM recruitment_data WHERE YearsOfExperience > 5;"
    },
    {
        "question": "What are the names of all male applicants?",
        "gold_query": "SELECT FirstName, LastName FROM recruitment_data WHERE Gender='Male';"
    },
    {
        "question": "What are the job titles of applications from California?",
        "gold_query": "SELECT Title FROM recruitment_data WHERE State='CA';"
    },
    {
        "question": "How many applicants applied on 15th of Jan, 2024?",
        "gold_query": "SELECT COUNT(*) FROM recruitment_data WHERE ApplicationDate='15-Jan-24';"
    },
    {
        "question": "How many training programs were conducted at Brandonview?",
        "gold_query": "SELECT COUNT(*) FROM training_and_development_data WHERE Location='Brandonview';"
    },
    {
        "question": "What are the training programs with a duration of more than 5 days?",
        "gold_query": "SELECT TrainingProgramName FROM training_and_development_data WHERE TrainingDurationInDays > 5;"
    },
    {
        "question": "What was the total cost of training programs conducted in 2023?",
        "gold_query": "SELECT SUM(TrainingCost) FROM training_and_development_data WHERE TrainingDate LIKE '%23';"
    },
    {
        "question": "What are the names of trainers who are from an external training program?",
        "gold_query": "SELECT Trainer FROM training_and_development_data WHERE TrainingType='External';"
    },
    {
        "question": "What is the training outcome of the employee with ID 150?",
        "gold_query": "SELECT TrainingOutcome FROM training_and_development_data WHERE EmpID=150;"
    },
    {
        "question": "What is the engagement score of employee with ID 101?",
        "gold_query": "SELECT EngagementScore FROM employee_engagement_survey_data WHERE EmpID=101;"
    },
    {
        "question": "How many employees scored above 8 in satisfaction?",
        "gold_query": "SELECT COUNT(*) FROM employee_engagement_survey_data WHERE SatisfactionScore > 8;"
    },
    {
        "question": "What is the work-life balance scores of employees surveyed on the 10th of May, 2024?",
        "gold_query": "SELECT WorkLifeBalanceScore FROM employee_engagement_survey_data WHERE SurveyDate='10/5/2024';"
    },
    {
        "question": "What is the average engagement score for all employees?",
        "gold_query": "SELECT AVG(EngagementScore) FROM employee_engagement_survey_data;"
    },
    {
        "question": "What are the survey details for employees with engagement scores below 5?",
        "gold_query": "SELECT * FROM employee_engagement_survey_data WHERE EngagementScore < 5;"
    },
	{
		"question": "What are the names of employees who are either in the same department as Jane Doe or share the same title?",
		"gold_query": "SELECT FirstName, LastName FROM employee_data WHERE DepartmentType = (SELECT DepartmentType FROM employee_data WHERE FirstName = 'Jane' AND LastName = 'Doe') OR Title = (SELECT Title FROM employee_data WHERE FirstName = 'Jane' AND LastName = 'Doe');"
	},
	{
		"question": "What is the number of hires in each year?",
		"gold_query": "SELECT strftime('%Y', '20' || substr(StartDate, 8, 2) || '-' || CASE substr(StartDate, 4, 3) WHEN 'Jan' THEN '01' WHEN 'Feb' THEN '02' WHEN 'Mar' THEN '03' WHEN 'Apr' THEN '04' WHEN 'May' THEN '05' WHEN 'Jun' THEN '06' WHEN 'Jul' THEN '07' WHEN 'Aug' THEN '08' WHEN 'Sep' THEN '09' WHEN 'Oct' THEN '10' WHEN 'Nov' THEN '11' WHEN 'Dec' THEN '12' END || '-' || substr(StartDate, 1, 2)) AS HireYear, COUNT(*) AS EmployeeCount FROM employee_data WHERE EmployeeStatus = 'Active' AND StartDate IS NOT NULL AND StartDate != '' GROUP BY strftime('%Y', '20' || substr(StartDate, 8, 2) || '-' || CASE substr(StartDate, 4, 3) WHEN 'Jan' THEN '01' WHEN 'Feb' THEN '02' WHEN 'Mar' THEN '03' WHEN 'Apr' THEN '04' WHEN 'May' THEN '05' WHEN 'Jun' THEN '06' WHEN 'Jul' THEN '07' WHEN 'Aug' THEN '08' WHEN 'Sep' THEN '09' WHEN 'Oct' THEN '10' WHEN 'Nov' THEN '11' WHEN 'Dec' THEN '12' END || '-' || substr(StartDate, 1, 2));"
	},
	{
		"question": "Who are the employees who have completed a training program lasting more than 3 days and with a cost above $500?",
		"gold_query": "SELECT e.FirstName, e.LastName, t.TrainingProgramName FROM employee_data e JOIN training_and_development_data t ON e.EmpID = t.EmpID WHERE t.TrainingDurationInDays > 3 AND t.TrainingCost > 500;"
	},
	{
		"question": "Who are the employees who have not attended any training program in the last two years?",
		"gold_query": "SELECT FirstName, LastName FROM employee_data WHERE EmpID NOT IN (SELECT EmpID FROM training_and_development_data WHERE TrainingDate > date('now', '-2 years'));"
	},
	{
		"question": "Who are the employees with the lowest engagement score in their department?",
		"gold_query": "SELECT FirstName, LastName, DepartmentType, EngagementScore FROM employee_engagement_survey_data ees JOIN employee_data ed ON ees.EmpID = ed.EmpID WHERE EngagementScore = (SELECT MIN(EngagementScore) FROM employee_engagement_survey_data WHERE DepartmentType = ed.DepartmentType);"
	},
	{
		"question": "What is the average cost and total number of training programs for each type?",
		"gold_query": "SELECT TrainingType, AVG(TrainingCost) AS AvgCost, COUNT(*) AS ProgramCount FROM training_and_development_data GROUP BY TrainingType;"
	},
	{
		"question": "Who are the employees whose last training program was more than a year ago?",
		"gold_query": "SELECT FirstName, LastName FROM employee_data WHERE EmpID IN (SELECT EmpID FROM training_and_development_data GROUP BY EmpID HAVING MAX(TrainingDate < date('now', '-2 year')));"
	},
	{
		"question": "Who are the names of employees who attended all training programs conducted in 2023?",
		"gold_query": "SELECT FirstName, LastName FROM employee_data WHERE NOT EXISTS (SELECT 1 FROM training_and_development_data t1 WHERE TrainingDate LIKE '%23' AND t1.TrainingType NOT IN (SELECT TrainingType FROM training_and_development_data t2 WHERE t2.EmpID = employee_data.EmpID));"
	},
	{
		"question": "What is the difference between the highest and lowest training costs for each year?",
		"gold_query": "SELECT strftime('%Y', '20' || substr(TrainingDate, 8, 2) || '-' || CASE substr(TrainingDate, 4, 3) WHEN 'Jan' THEN '01' WHEN 'Feb' THEN '02' WHEN 'Mar' THEN '03' WHEN 'Apr' THEN '04' WHEN 'May' THEN '05' WHEN 'Jun' THEN '06' WHEN 'Jul' THEN '07' WHEN 'Aug' THEN '08' WHEN 'Sep' THEN '09' WHEN 'Oct' THEN '10' WHEN 'Nov' THEN '11' WHEN 'Dec' THEN '12' END || '-' || substr(TrainingDate, 1, 2)) AS TrainingYear, MAX(TrainingCost) - MIN(TrainingCost) AS CostDifference FROM training_and_development_data GROUP BY strftime('%Y', '20' || substr(TrainingDate, 8, 2) || '-' || CASE substr(TrainingDate, 4, 3) WHEN 'Jan' THEN '01' WHEN 'Feb' THEN '02' WHEN 'Mar' THEN '03' WHEN 'Apr' THEN '04' WHEN 'May' THEN '05' WHEN 'Jun' THEN '06' WHEN 'Jul' THEN '07' WHEN 'Aug' THEN '08' WHEN 'Sep' THEN '09' WHEN 'Oct' THEN '10' WHEN 'Nov' THEN '11' WHEN 'Dec' THEN '12' END || '-' || substr(TrainingDate, 1, 2));"
	},
	{
		"question": "Who are the employees who have completed the internal training program?",
		"gold_query": "SELECT DISTINCT e.FirstName, e.LastName FROM employee_data e JOIN training_and_development_data t1 ON e.EmpID = t1.EmpID WHERE t1.TrainingType = 'Internal';"
	},
	{
		"question": "What is the average satisfaction score for employees by tenure in 5-year increments?",
		"gold_query": "SELECT FLOOR((julianday(DATE('now')) - julianday(CASE WHEN LENGTH(StartDate) = 9 THEN '20' || substr(StartDate, 8, 2) || '-' || CASE substr(StartDate, 4, 3) WHEN 'Jan' THEN '01' WHEN 'Feb' THEN '02' WHEN 'Mar' THEN '03' WHEN 'Apr' THEN '04' WHEN 'May' THEN '05' WHEN 'Jun' THEN '06' WHEN 'Jul' THEN '07' WHEN 'Aug' THEN '08' WHEN 'Sep' THEN '09' WHEN 'Oct' THEN '10' WHEN 'Nov' THEN '11' WHEN 'Dec' THEN '12' END || '-' || substr(StartDate, 1, 2) ELSE StartDate END)) / 365 / 5) * 5 AS TenureGroup, AVG(SatisfactionScore) AS AvgSatisfaction FROM employee_data ed JOIN employee_engagement_survey_data ees ON ed.EmpID = ees.EmpID WHERE StartDate IS NOT NULL AND SatisfactionScore IS NOT NULL GROUP BY TenureGroup ORDER BY TenureGroup;"
	},
	{
		"question": "What are the names of employees who have taken training at the same location more than twice?",
		"gold_query": "SELECT FirstName, LastName FROM employee_data e WHERE EmpID IN (SELECT EmpID FROM training_and_development_data GROUP BY EmpID, Location HAVING COUNT(*) > 2);"
	}
]

evaluation_set[0]

{'question': 'What are the names of employees working in sales?',
 'gold_query': "SELECT FirstName, LastName FROM employee_data WHERE BusinessUnit='Sales';"}

In [31]:
table_schemas = """
-- Employee Data Table
CREATE TABLE employee_data (
    EmpID INTEGER,
    FirstName TEXT,
    LastName TEXT,
    StartDate TEXT,
    ExitDate TEXT,
    Title TEXT,
    Supervisor TEXT,
    ADEmail TEXT,
    BusinessUnit TEXT,
    EmployeeStatus TEXT,
    EmployeeType TEXT,
    PayZone TEXT,
    EmployeeClassificationType TEXT,
    TerminationType TEXT,
    TerminationDescription TEXT,
    DepartmentType TEXT,
    Division TEXT,
    DOB TEXT,
    State TEXT,
    JobFunctionDescription TEXT,
    GenderCode TEXT,
    LocationCode INTEGER,
    RaceDesc TEXT,
    MaritalDesc TEXT,
    PerformanceScore TEXT,
    CurrentEmployeeRating INTEGER
);

-- Recruitment Data Table
CREATE TABLE recruitment_data (
    ApplicantID INTEGER,
    ApplicationDate TEXT,
    FirstName TEXT,
    LastName TEXT,
    Gender TEXT,
    DOB TEXT,
    PhoneNumber TEXT,
    Email TEXT,
    Address TEXT,
    City TEXT,
    State TEXT,
    ZipCode INTEGER,
    Country TEXT,
    EducationLevel TEXT,
    YearsofExperience INTEGER,
    DesiredSalary REAL,
    Title TEXT,
    Status TEXT
);

-- Training and Development Data Table
CREATE TABLE training_and_development_data (
    EmpID INTEGER,
    TrainingDate TEXT,
    TrainingProgramName TEXT,
    TrainingType TEXT,
    TrainingOutcome TEXT,
    Location TEXT,
    Trainer TEXT,
    TrainingDurationInDays INTEGER,
    TrainingCost INTEGER
);

-- Employee Engagement Survey Data Table
CREATE TABLE employee_engagement_survey_data (
    EmpID INTEGER,
    SurveyDate TEXT,
    EngagementScore INTEGER,
    SatisfactionScore INTEGER,
    WorklifeBalanceScore INTEGER
);
"""

In [32]:
# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()

In [33]:
processed_dir  = "Processed Data"
def sqlalchemy_engine(region:str):
    """Create a SQLAlchemy engine for the given region"""
    assert region in os.listdir(processed_dir), f"{region} is not a valid region from {os.listdir(processed_dir)}"
    # Create a SQLAlchemy database for each region
    engine = create_engine(f"sqlite:///{region}.db")
    metadata_obj = MetaData()
    region_path = os.path.join(processed_dir,region)
    dfs = []
    for dataframes_path in os.listdir(region_path):
        if dataframes_path.endswith(".csv"):
            df = pd.read_csv(os.path.join(region_path,dataframes_path),index_col=False)
            dfs.append((dataframes_path,df))
    pbar = tqdm(total=len(dfs),desc=f"Creating tables for {region}")
    for _, df_table_name in enumerate(dfs):
        table_name = df_table_name[0]
        table_name = table_name.split(".")[0]
        df = df_table_name[1]
        # print(f"Creating table: {table_name}")
        create_table_from_dataframe(df,table_name, engine, metadata_obj)
        # print(f"Done creating table for: {table_name}")
        pbar.update(1)
    return engine

In [9]:
# us_engine = sqlalchemy_engine("US")
india_engine = sqlalchemy_engine("India")
# china_engine = sqlalchemy_engine("China")
# europe_engine = sqlalchemy_engine("Europe")
# global_engine = sqlalchemy_engine("Global")
# aus_nz_canada_engine = sqlalchemy_engine("AUS_NZ_CANADA")
# japan_engine = sqlalchemy_engine("Japan")
# emerging_engine = sqlalchemy_engine("Emerging")

Creating tables for India: 100%|███████████████████████████████████████████████████████| 29/29 [00:01<00:00, 16.96it/s]


In [10]:
def get_table_infos(sql_engine:sqlalchemy.engine.base.Engine,region:str):
    """Get all the tables info in the database based on the given region"""
    inspector = inspect(sql_engine)
    table_names = inspector.get_table_names()
    table_infos_dict = {tb: [] for tb in table_names}
    for tb in table_names:
        column_dict = inspector.get_columns(tb)
        schema_str = ""
        primary_keys = []
        for col in column_dict:
            schema_str += f"{col['name']} ({col['type']}), "
            if col["primary_key"] not in primary_keys:
                primary_keys.append(col["name"])
        
        with open(os.path.join(processed_dir,region,f"{tb}.json")) as f:
            table_info = json.loads(f.read())
        table_info = list(table_info.items())
        # print(table_info)
        table_infos_dict[tb] = [
            {
                "table_info": f"CREATE TABLE {tb} { {schema_str[:-2]}};",
                "table_summary": f'{table_info[0][-1]}. {table_info[-1][-1]}. ',
            }
        ]
    return table_infos_dict

In [11]:
india_tb_dict = get_table_infos(india_engine,"India")
india_tb_dict

{'DollarIndia': [{'table_info': "CREATE TABLE DollarIndia {'Industry_Name (VARCHAR), Number_of_firms (INTEGER), Average_Company_Age_years_ (INTEGER), Market_Cap_millions_ (INTEGER), Book_Equity_millions_ (INTEGER), Enteprise_Value_millions_ (INTEGER), Invested_Capital_millions_ (INTEGER), Total_Debt_including_leases_millions_ (INTEGER), Revenues_millions_ (INTEGER), Gross_Profit_millions_ (INTEGER), EBITDA_millions_ (INTEGER), EBIT_Operating_Income_millions_ (INTEGER), Net_Income_millions_ (INTEGER)'};",
   'table_summary': 'To report aggregated dollar value of key operating and marker numbers, by industry group, in millions of US $.. This table provides a comprehensive overview of various industries, detailing key financial metrics such as the number of firms, average company age, market capitalization, and profitability indicators. It serves as a valuable resource for understanding the economic landscape and comparing the financial health of different sectors.. '}],
 'EVAIndia': [{'t

In [34]:
import random

random.Random(0).shuffle(evaluation_set)
trainset, devset, testset = evaluation_set[:20], evaluation_set[20:25], evaluation_set[25:32]

len(trainset), len(devset), len(testset)

print(trainset)

[{'question': 'How many employees are currently active?', 'gold_query': "SELECT COUNT(*) FROM employee_data WHERE EmployeeStatus='Active';"}, {'question': 'What are the names of all male applicants?', 'gold_query': "SELECT FirstName, LastName FROM recruitment_data WHERE Gender='Male';"}, {'question': 'Who are the employees who have completed the internal training program?', 'gold_query': "SELECT DISTINCT e.FirstName, e.LastName FROM employee_data e JOIN training_and_development_data t1 ON e.EmpID = t1.EmpID WHERE t1.TrainingType = 'Internal';"}, {'question': 'What are the names of employees who are either in the same department as Jane Doe or share the same title?', 'gold_query': "SELECT FirstName, LastName FROM employee_data WHERE DepartmentType = (SELECT DepartmentType FROM employee_data WHERE FirstName = 'Jane' AND LastName = 'Doe') OR Title = (SELECT Title FROM employee_data WHERE FirstName = 'Jane' AND LastName = 'Doe');"}, {'question': 'What are the survey details for employees w

In [6]:
!pip install sqlparse

In [12]:
import sqlite3
import re
import sqlparse
from tqdm import tqdm

def normalize_sql_string(sql_string):
    sql_query = re.sub(r'```sql\n|```', '', sql_string).strip()
    sql_query = sql_query.strip()
    sql_query = re.sub(r'\s+', ' ', sql_query)
    sql_query = re.sub(
        r'\b(select|from|where|count|as|and|or|in|not|is|like|group|order|by|having|limit|join)\b',
        lambda match: match.group(0).upper(),
        sql_query,
        flags=re.IGNORECASE,
    )
    sql_query = re.sub(r'\s*,\s*', ', ', sql_query)
    sql_query = re.sub(r'\s*\(\s*', '(', sql_query)
    sql_query = re.sub(r'\s*\)\s*', ')', sql_query)
    return sql_query

def execute_sql_query(db_connection, sql_query):
    """Executes the SQL query on the SQLite database and returns the result"""
    try:
        cursor = db_connection.cursor()
        cursor.execute(sql_query)
        result = cursor.fetchall()
        return result
    except Exception as e:
        print(f"Error executing SQL query: {e}")
        return None

def compare_sqls(ground_truth, prediction, db_connection):
    """Compare execution results of ground truth and predicted SQL queries."""
    ground_truth = normalize_sql_string(ground_truth)
    prediction = normalize_sql_string(prediction)

    # Execute both SQL queries on the database
    ground_truth_result = execute_sql_query(db_connection, ground_truth)
    prediction_result = execute_sql_query(db_connection, prediction)

    print('\nground truth sql query: ', ground_truth)
    print('ground truth result: ', ground_truth_result)
    print('prediction sql query: ', prediction)
    print('prediction result: ', prediction_result)

    # Compare results based on row count and values
    if ground_truth_result is not None and prediction_result is not None:
        return ground_truth_result == prediction_result
    return False


def compare_sqls_wrapper(example, prediction):
  """Wrapper for compare_sqls to access example data and establish db_connection."""
  db_connection = sqlite3.connect('employees.db')  # Establish connection inside the wrapper
  ground_truth = example['gold_query']  # Access ground truth from the example
  # Assuming prediction.sql contains the predicted SQL query
  return compare_sqls(ground_truth, prediction.sql, db_connection)



In [14]:
import dspy

class Text2SQLSignature(dspy.Signature):
	"""Transform a natural language query into a SQL query.
	You will be given a question which will tell what you need to do
	and a sql_context which will give some additional context to generate the right SQL.
	Only generate the SQL query nothing else. You should give one correct answer.
	starting and ending with ```
	"""

	question = dspy.InputField(desc="Natural language query")
	sql_context = dspy.InputField(desc="Context for the query")
	sql = dspy.OutputField(desc="SQL Query")

In [ ]:
generate_sql_from_query = dspy.Predict(signature=Text2SQLSignature)
result = generate_sql_from_query(
    question='What is the average age of employees in the Sales department?',
    # sql_context= '\n'.join(list(india_tb_dict))[:200]
    sql_context = table_schemas
)

print("Raw result:", result.sql)

In [27]:
class Text2SQLProgram(dspy.Module):
	def __init__(self):
		super().__init__()
		self.program = dspy.ChainOfThought(signature=Text2SQLSignature)

	def forward(self, question, sql_context):
		return self.program(
			question=question,
			sql_context=sql_context
		)

In [28]:
text2sql = Text2SQLProgram()

print(text2sql)

result = text2sql(question='How many employees are currently not active?', sql_context=table_schemas)

for k, v in result.items():
    print(f"\n{k.upper()}:\n")
    print(v)

print(result.sql)

program = Predict(StringSignature(question, sql_context -> reasoning, sql
    instructions='Transform a natural language query into a SQL query.\nYou will be given a question which will tell what you need to do\nand a sql_context which will give some additional context to generate the right SQL.\nOnly generate the SQL query nothing else. You should give one correct answer.\nstarting and ending with ```'
    question = Field(annotation=str required=True json_schema_extra={'desc': 'Natural language query', '__dspy_field_type': 'input', 'prefix': 'Question:'})
    sql_context = Field(annotation=str required=True json_schema_extra={'desc': 'Context for the query', '__dspy_field_type': 'input', 'prefix': 'Sql Context:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    sql = Field(annotation=str required=True json_schema_extra={'desc': 'SQL Query', '_

In [35]:
db_connection = sqlite3.connect('employees.db')

scores = []

for x in tqdm(testset, total=len(testset)):
    # Assuming text2sql function is available and works correctly
    prediction = text2sql(question=x['question'], sql_context=table_schemas)
    ground_truth = x['gold_query']

    # Compare SQL execution results
    score = compare_sqls(ground_truth=ground_truth, prediction=prediction.sql, db_connection=db_connection)

    scores.append(score)

# Print the final score
print(f"\nExecution Accuracy: {sum(scores)}/{len(scores)}")

100%|████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 97.05it/s]


ground truth sql query:  SELECT COUNT(*)FROM employee_engagement_survey_data WHERE SatisfactionScore > 8;
ground truth result:  [(0,)]
prediction sql query:  SELECT COUNT(*)AS NumberOfEmployees FROM employee_engagement_survey_data WHERE SatisfactionScore > 8;
prediction result:  [(0,)]

ground truth sql query:  SELECT Title FROM recruitment_data WHERE State='CA';
ground truth result:  [('Public relations officer',)]
prediction sql query:  SELECT DISTINCT Title FROM recruitment_data WHERE State = 'California';
prediction result:  []

ground truth sql query:  SELECT ADEmail FROM employee_data WHERE EmpID=200;
ground truth result:  []
prediction sql query:  SELECT ADEmail FROM employee_data WHERE EmpID = 200;
prediction result:  []

ground truth sql query:  SELECT Trainer FROM training_and_development_data WHERE TrainingType='External';
ground truth result:  [('Yolanda Wilson',), ('Lori Mckee',), ('Hannah Smith',), ('Christine Stevens',), ('Bonnie King',), ('Jennifer Cline MD',), ('Josep

In [36]:
from dspy.teleprompt import LabeledFewShot
from dspy import Evaluate

trainset = [dspy.Example(sql_context = table_schemas, **d).with_inputs('question', 'sql_context') for d in trainset]
devset = [dspy.Example(sql_context = table_schemas, **d).with_inputs('question', 'sql_context') for d in devset]

text2sql = Text2SQLProgram()

def compare_sqls_wrapper_with_trace(example, prediction, trace=None):  # Add trace argument
    """Wrapper for compare_sqls to access example data and establish db_connection."""
    db_connection = sqlite3.connect('employees.db')  # Establish connection inside the wrapper
    ground_truth = example['gold_query']  # Access ground truth from the example
    # Assuming prediction.sql contains the predicted SQL query
    return compare_sqls(ground_truth, prediction.sql, db_connection)

# k = number of few shot examples
optimizer = LabeledFewShot(k=4)

optimized_text2sql = optimizer.compile(
    student=text2sql,
    trainset=trainset,
)

evaluate = Evaluate(
    devset=devset,
    metric=compare_sqls_wrapper_with_trace,
    num_threads=3,
    display_progress=True,
    display_table=0
)

# Get the evaluation results
results = evaluate(optimized_text2sql)

# Access the individual scores (assuming they are stored in a 'scores' attribute)
scores = results

print(scores)

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]
ground truth sql query:  SELECT TrainingProgramName FROM training_and_development_data WHERE TrainingDurationInDays > 5;
ground truth result:  []
prediction sql query:  SELECT DISTINCT TrainingProgramName FROM training_and_development_data WHERE TrainingDurationInDays > 5;
prediction result:  []
Error executing SQL query: no such function: FLOOR
Average Metric: 1.00 / 1 (100.0%):   0%|                                                         | 0/5 [00:00<?, ?it/s]
ground truth sql query:  SELECT FLOOR((julianday(DATE('now'))- julianday(CASE WHEN LENGTH(StartDate)= 9 THEN '20' || substr(StartDate, 8, 2)|| '-' || CASE substr(StartDate, 4, 3)WHEN 'Jan' THEN '01' WHEN 'Feb' THEN '02' WHEN 'Mar' THEN '03' WHEN 'Apr' THEN '04' WHEN 'May' THEN '05' WHEN 'Jun' THEN '06' WHEN 'Jul' THEN '07' WHEN 'Aug' THEN '08' WHEN 'Sep' THEN '09' WHEN 'Oct' THEN '10' WHEN 'Nov' THEN '11' WHE

2025/01/28 13:22:33 ERROR dspy.utils.parallelizer: Error processing item Example({'sql_context': '\n-- Employee Data Table\nCREATE TABLE employee_data (\n    EmpID INTEGER,\n    FirstName TEXT,\n    LastName TEXT,\n    StartDate TEXT,\n    ExitDate TEXT,\n    Title TEXT,\n    Supervisor TEXT,\n    ADEmail TEXT,\n    BusinessUnit TEXT,\n    EmployeeStatus TEXT,\n    EmployeeType TEXT,\n    PayZone TEXT,\n    EmployeeClassificationType TEXT,\n    TerminationType TEXT,\n    TerminationDescription TEXT,\n    DepartmentType TEXT,\n    Division TEXT,\n    DOB TEXT,\n    State TEXT,\n    JobFunctionDescription TEXT,\n    GenderCode TEXT,\n    LocationCode INTEGER,\n    RaceDesc TEXT,\n    MaritalDesc TEXT,\n    PerformanceScore TEXT,\n    CurrentEmployeeRating INTEGER\n);\n\n-- Recruitment Data Table\nCREATE TABLE recruitment_data (\n    ApplicantID INTEGER,\n    ApplicationDate TEXT,\n    FirstName TEXT,\n    LastName TEXT,\n    Gender TEXT,\n    DOB TEXT,\n    PhoneNumber TEXT,\n    Email T

Average Metric: 2.00 / 4 (50.0%): 100%|██████████████████████████████████████████████████| 5/5 [01:02<00:00, 12.53s/it]

2025/01/28 13:22:33 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 5 (40.0%)



40.0


In [41]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

from dspy import *

trainset = [dspy.Example(**d).with_inputs('question', 'sql_context') for d in trainset]
devset = [dspy.Example(**d).with_inputs('question', 'sql_context') for d in devset]

text2sql = Text2SQLProgram()

def compare_sqls_wrapper_with_trace(example, prediction, trace=None):  # Add trace argument
    """Wrapper for compare_sqls to access example data and establish db_connection."""
    db_connection = sqlite3.connect('employees.db')  # Establish connection inside the wrapper
    ground_truth = example['gold_query']  # Access ground truth from the example
    # Assuming prediction.sql contains the predicted SQL query
    return compare_sqls(ground_truth, prediction.sql, db_connection)

config = dict(max_bootstrapped_demos=4, max_labeled_demos=4, num_candidate_programs=10, num_threads=4)
teleprompter = BootstrapFewShotWithRandomSearch(metric=compare_sqls_wrapper_with_trace, **config)

optimized_program = teleprompter.compile(
    text2sql,
    trainset=trainset[:5],
  )


Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 10 candidate sets.
  0%|                                                                                           | 0/20 [00:00<?, ?it/s]
ground truth sql query:  SELECT COUNT(*)FROM employee_data WHERE EmployeeStatus='Active';
ground truth result:  [(51,)]
prediction sql query:  SELECT COUNT(*)AS ActiveEmployeeCount FROM employee_data WHERE EmployeeStatus = 'Active';
prediction result:  [(51,)]
Average Metric: 1.00 / 1 (100.0%):   0%|                                                        | 0/20 [00:00<?, ?it/s]
ground truth sql query:  SELECT DISTINCT e.FirstName, e.LastName FROM employee_data e JOIN training_and_development_data t1 ON e.EmpID = t1.EmpID WHERE t1.TrainingType = 'Internal';
ground truth result:  [('Ellie', 'Holder')]
prediction sql query:  SELECT e.EmpID, e.FirstName, e.LastName FROM employee_data e JOIN training_and_development_data t ON e.EmpID = t.EmpID WHERE t.TrainingOutcome = 'Compl

2025/01/28 13:28:04 ERROR dspy.utils.parallelizer: Error processing item Example({'sql_context': '\n-- Employee Data Table\nCREATE TABLE employee_data (\n    EmpID INTEGER,\n    FirstName TEXT,\n    LastName TEXT,\n    StartDate TEXT,\n    ExitDate TEXT,\n    Title TEXT,\n    Supervisor TEXT,\n    ADEmail TEXT,\n    BusinessUnit TEXT,\n    EmployeeStatus TEXT,\n    EmployeeType TEXT,\n    PayZone TEXT,\n    EmployeeClassificationType TEXT,\n    TerminationType TEXT,\n    TerminationDescription TEXT,\n    DepartmentType TEXT,\n    Division TEXT,\n    DOB TEXT,\n    State TEXT,\n    JobFunctionDescription TEXT,\n    GenderCode TEXT,\n    LocationCode INTEGER,\n    RaceDesc TEXT,\n    MaritalDesc TEXT,\n    PerformanceScore TEXT,\n    CurrentEmployeeRating INTEGER\n);\n\n-- Recruitment Data Table\nCREATE TABLE recruitment_data (\n    ApplicantID INTEGER,\n    ApplicationDate TEXT,\n    FirstName TEXT,\n    LastName TEXT,\n    Gender TEXT,\n    DOB TEXT,\n    PhoneNumber TEXT,\n    Email T

Average Metric: 4.00 / 6 (66.7%):  35%|█████████████████▏                               | 7/20 [02:04<05:11, 23.93s/it]
ground truth sql query:  SELECT WorkLifeBalanceScore FROM employee_engagement_survey_data WHERE SurveyDate='10/10/2024';
ground truth result:  []
prediction sql query:  SELECT WorklifeBalanceScore FROM employee_engagement_survey_data WHERE SurveyDate = '2024-05-10';
prediction result:  []
Average Metric: 5.00 / 7 (71.4%):  35%|█████████████████▏                               | 7/20 [02:04<05:11, 23.93s/it]

2025/01/28 13:29:02 ERROR dspy.utils.parallelizer: Error processing item Example({'sql_context': '\n-- Employee Data Table\nCREATE TABLE employee_data (\n    EmpID INTEGER,\n    FirstName TEXT,\n    LastName TEXT,\n    StartDate TEXT,\n    ExitDate TEXT,\n    Title TEXT,\n    Supervisor TEXT,\n    ADEmail TEXT,\n    BusinessUnit TEXT,\n    EmployeeStatus TEXT,\n    EmployeeType TEXT,\n    PayZone TEXT,\n    EmployeeClassificationType TEXT,\n    TerminationType TEXT,\n    TerminationDescription TEXT,\n    DepartmentType TEXT,\n    Division TEXT,\n    DOB TEXT,\n    State TEXT,\n    JobFunctionDescription TEXT,\n    GenderCode TEXT,\n    LocationCode INTEGER,\n    RaceDesc TEXT,\n    MaritalDesc TEXT,\n    PerformanceScore TEXT,\n    CurrentEmployeeRating INTEGER\n);\n\n-- Recruitment Data Table\nCREATE TABLE recruitment_data (\n    ApplicantID INTEGER,\n    ApplicationDate TEXT,\n    FirstName TEXT,\n    LastName TEXT,\n    Gender TEXT,\n    DOB TEXT,\n    PhoneNumber TEXT,\n    Email T

Average Metric: 5.00 / 7 (71.4%):  45%|██████████████████████                           | 9/20 [03:02<04:42, 25.68s/it]

...

2025/01/28 13:33:05 ERROR dspy.utils.parallelizer: Error processing item Example({'sql_context': '\n-- Employee Data Table\nCREATE TABLE employee_data (\n    EmpID INTEGER,\n    FirstName TEXT,\n    LastName TEXT,\n    StartDate TEXT,\n    ExitDate TEXT,\n    Title TEXT,\n    Supervisor TEXT,\n    ADEmail TEXT,\n    BusinessUnit TEXT,\n    EmployeeStatus TEXT,\n    EmployeeType TEXT,\n    PayZone TEXT,\n    EmployeeClassificationType TEXT,\n    TerminationType TEXT,\n    TerminationDescription TEXT,\n    DepartmentType TEXT,\n    Division TEXT,\n    DOB TEXT,\n    State TEXT,\n    JobFunctionDescription TEXT,\n    GenderCode TEXT,\n    LocationCode INTEGER,\n    RaceDesc TEXT,\n    MaritalDesc TEXT,\n    PerformanceScore TEXT,\n    CurrentEmployeeRating INTEGER\n);\n\n-- Recruitment Data Table\nCREATE TABLE recruitment_data (\n    ApplicantID INTEGER,\n    ApplicationDate TEXT,\n    FirstName TEXT,\n    LastName TEXT,\n    Gender TEXT,\n    DOB TEXT,\n    PhoneNumber TEXT,\n    Email T

Average Metric: 8.00 / 10 (80.0%):  90%|██████████████████████████████████████████▎    | 18/20 [07:05<00:51, 25.52s/it]

2025/01/28 13:34:05 ERROR dspy.utils.parallelizer: Error processing item Example({'sql_context': '\n-- Employee Data Table\nCREATE TABLE employee_data (\n    EmpID INTEGER,\n    FirstName TEXT,\n    LastName TEXT,\n    StartDate TEXT,\n    ExitDate TEXT,\n    Title TEXT,\n    Supervisor TEXT,\n    ADEmail TEXT,\n    BusinessUnit TEXT,\n    EmployeeStatus TEXT,\n    EmployeeType TEXT,\n    PayZone TEXT,\n    EmployeeClassificationType TEXT,\n    TerminationType TEXT,\n    TerminationDescription TEXT,\n    DepartmentType TEXT,\n    Division TEXT,\n    DOB TEXT,\n    State TEXT,\n    JobFunctionDescription TEXT,\n    GenderCode TEXT,\n    LocationCode INTEGER,\n    RaceDesc TEXT,\n    MaritalDesc TEXT,\n    PerformanceScore TEXT,\n    CurrentEmployeeRating INTEGER\n);\n\n-- Recruitment Data Table\nCREATE TABLE recruitment_data (\n    ApplicantID INTEGER,\n    ApplicationDate TEXT,\n    FirstName TEXT,\n    LastName TEXT,\n    Gender TEXT,\n    DOB TEXT,\n    PhoneNumber TEXT,\n    Email T

Average Metric: 8.00 / 10 (80.0%):  95%|████████████████████████████████████████████▋  | 19/20 [08:05<00:32, 32.10s/it]

RateLimitError: litellm.RateLimitError: RateLimitError: OpenAIException - Error code: 429 - {'error': {'code': 'RateLimitReached', 'message': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 77210 seconds before retrying.', 'details': 'Rate limit of 50 per 86400s exceeded for UserByModelByDay. Please wait 77210 seconds before retrying.'}}

In [44]:
optimized_program.save('fewshot_optimized_text2sql_model.json')

In [39]:
class Correctness(dspy.Signature):
    """Assess if the SQL query accurately answers the given natural language query based on the provided context."""

    sql_prompt = dspy.InputField(desc="Natural language query ")
    sql_context = dspy.InputField(desc="Context for the query")
    sql = dspy.InputField(desc="SQL query")
    correct = dspy.OutputField(desc="Indicate whether the SQL query correctly answers the natural language query based on the given context", prefix="Yes/No:")

In [40]:
def correctness_metric(example, pred, trace=None):
    sql_prompt, sql_context, sql = example.sql_prompt, example.sql_context, pred.sql

    correctness = dspy.Predict(Correctness)

    with dspy.context(lm=evaluator_lm): 
        correct = correctness(
            sql_prompt=sql_prompt,
            sql_context=sql_context,
            sql=sql,
        )
    
    score = int(correct.correct=="Yes")

    if trace is not None:
        return score == 1

    return score

In [ ]:
class FactJudge(dspy.Signature):
    """Judge if the answer is factually correct based on the context."""

    sql_context = dspy.InputField(desc="Context for the prediction")
    question = dspy.InputField(desc="Question to be answered")
    answer = dspy.InputField(desc="Answer for the question")
    factually_correct = dspy.OutputField(desc="Is the answer factually correct based on the context?", prefix="Factual[Yes/No]:")

judge = dspy.ChainOfThought(FactJudge)

def factuality_metric(example, pred):
    factual = judge(context=example.context, question=example.question, answer=pred.answer)
    return int(factual=="Yes")